In [51]:
import pandas as pd
import numpy as np

In [52]:
import warnings

warnings.filterwarnings("ignore")

In [53]:
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [54]:
df = pd.read_csv("train.csv", encoding="utf-8")

In [55]:
df

,Cluster/Country/Region,SubCluster,calweek,brand,dc,product_code,customer_code,Quantity
0,US,SC1,202001,BRANDA,DC1,PRDCT1,CUSTA,537
1,US,SC1,202002,BRANDA,DC1,PRDCT1,CUSTA,544
2,US,SC1,202003,BRANDA,DC1,PRDCT1,CUSTA,574
3,US,SC1,202004,BRANDA,DC1,PRDCT1,CUSTA,523
4,US,SC1,202005,BRANDA,DC1,PRDCT1,CUSTA,571
...,...,...,...,...,...,...,...,...
18463,US,SC2,202311,BRANDC,DC4,PRDCT9,CUSTZ,1753
18464,US,SC2,202312,BRANDC,DC4,PRDCT9,CUSTZ,1752
18465,US,SC2,202313,BRANDC,DC4,PRDCT9,CUSTZ,1673
18466,US,SC2,202314,BRANDC,DC4,PRDCT9,CUSTZ,1752


In [56]:
df.columns

Index(['Cluster/Country/Region', 'SubCluster', 'calweek', 'brand', 'dc',
       'product_code', 'customer_code', 'Quantity'],
      dtype='object')

In [57]:
df1 = df.drop(['Cluster/Country/Region', 'SubCluster', 'brand', 'dc'], axis = 1)

In [58]:
df1

,calweek,product_code,customer_code,Quantity
0,202001,PRDCT1,CUSTA,537
1,202002,PRDCT1,CUSTA,544
2,202003,PRDCT1,CUSTA,574
3,202004,PRDCT1,CUSTA,523
4,202005,PRDCT1,CUSTA,571
...,...,...,...,...
18463,202311,PRDCT9,CUSTZ,1753
18464,202312,PRDCT9,CUSTZ,1752
18465,202313,PRDCT9,CUSTZ,1673
18466,202314,PRDCT9,CUSTZ,1752


In [59]:
df1['timeseries_key'] = df['product_code'] + '_' + df['customer_code']

In [60]:
df1.set_index('calweek',inplace=True)

In [61]:
df1['timeseries_key'].unique()

array(['PRDCT1_CUSTA', 'PRDCT1_CUSTB', 'PRDCT1_CUSTC', 'PRDCT1_CUSTD',
       'PRDCT1_CUSTE', 'PRDCT1_CUSTF', 'PRDCT2_CUSTA', 'PRDCT2_CUSTB',
       'PRDCT2_CUSTC', 'PRDCT2_CUSTD', 'PRDCT2_CUSTE', 'PRDCT2_CUSTF',
       'PRDCT3_CUSTA', 'PRDCT3_CUSTB', 'PRDCT3_CUSTC', 'PRDCT3_CUSTD',
       'PRDCT3_CUSTE', 'PRDCT3_CUSTF', 'PRDCT4_CUSTA', 'PRDCT4_CUSTB',
       'PRDCT4_CUSTC', 'PRDCT4_CUSTD', 'PRDCT4_CUSTE', 'PRDCT4_CUSTF',
       'PRDCT5_CUSTA', 'PRDCT5_CUSTB', 'PRDCT5_CUSTC', 'PRDCT5_CUSTD',
       'PRDCT5_CUSTE', 'PRDCT5_CUSTF', 'PRDCT6_CUSTA', 'PRDCT6_CUSTB',
       'PRDCT6_CUSTC', 'PRDCT6_CUSTD', 'PRDCT6_CUSTE', 'PRDCT6_CUSTF',
       'PRDCT7_CUSTA', 'PRDCT7_CUSTB', 'PRDCT7_CUSTC', 'PRDCT7_CUSTD',
       'PRDCT7_CUSTE', 'PRDCT7_CUSTF', 'PRDCT8_CUSTA', 'PRDCT8_CUSTB',
       'PRDCT8_CUSTC', 'PRDCT8_CUSTD', 'PRDCT8_CUSTE', 'PRDCT8_CUSTF',
       'PRDCT9_CUSTA', 'PRDCT9_CUSTB', 'PRDCT9_CUSTC', 'PRDCT9_CUSTD',
       'PRDCT9_CUSTE', 'PRDCT9_CUSTF', 'PRDCT1_CUSTK', 'PRDCT1_CUSTL',
      

In [62]:
timeseries_key_values= ['PRDCT1_CUSTA', 'PRDCT1_CUSTB', 'PRDCT1_CUSTC', 'PRDCT1_CUSTD',
       'PRDCT1_CUSTE', 'PRDCT1_CUSTF', 'PRDCT2_CUSTA', 'PRDCT2_CUSTB',
       'PRDCT2_CUSTC', 'PRDCT2_CUSTD', 'PRDCT2_CUSTE', 'PRDCT2_CUSTF',
       'PRDCT3_CUSTA', 'PRDCT3_CUSTB', 'PRDCT3_CUSTC', 'PRDCT3_CUSTD',
       'PRDCT3_CUSTE', 'PRDCT3_CUSTF', 'PRDCT4_CUSTA', 'PRDCT4_CUSTB',
       'PRDCT4_CUSTC', 'PRDCT4_CUSTD', 'PRDCT4_CUSTE', 'PRDCT4_CUSTF',
       'PRDCT5_CUSTA', 'PRDCT5_CUSTB', 'PRDCT5_CUSTC', 'PRDCT5_CUSTD',
       'PRDCT5_CUSTE', 'PRDCT5_CUSTF', 'PRDCT6_CUSTA', 'PRDCT6_CUSTB',
       'PRDCT6_CUSTC', 'PRDCT6_CUSTD', 'PRDCT6_CUSTE', 'PRDCT6_CUSTF',
       'PRDCT7_CUSTA', 'PRDCT7_CUSTB', 'PRDCT7_CUSTC', 'PRDCT7_CUSTD',
       'PRDCT7_CUSTE', 'PRDCT7_CUSTF', 'PRDCT8_CUSTA', 'PRDCT8_CUSTB',
       'PRDCT8_CUSTC', 'PRDCT8_CUSTD', 'PRDCT8_CUSTE', 'PRDCT8_CUSTF',
       'PRDCT9_CUSTA', 'PRDCT9_CUSTB', 'PRDCT9_CUSTC', 'PRDCT9_CUSTD',
       'PRDCT9_CUSTE', 'PRDCT9_CUSTF', 'PRDCT1_CUSTK', 'PRDCT1_CUSTL',
       'PRDCT1_CUSTM', 'PRDCT1_CUSTX', 'PRDCT1_CUSTY', 'PRDCT1_CUSTZ',
       'PRDCT2_CUSTK', 'PRDCT2_CUSTL', 'PRDCT2_CUSTM', 'PRDCT2_CUSTX',
       'PRDCT2_CUSTY', 'PRDCT2_CUSTZ', 'PRDCT3_CUSTK', 'PRDCT3_CUSTL',
       'PRDCT3_CUSTM', 'PRDCT3_CUSTX', 'PRDCT3_CUSTY', 'PRDCT3_CUSTZ',
       'PRDCT4_CUSTK', 'PRDCT4_CUSTL', 'PRDCT4_CUSTM', 'PRDCT4_CUSTX',
       'PRDCT4_CUSTY', 'PRDCT4_CUSTZ', 'PRDCT5_CUSTK', 'PRDCT5_CUSTL',
       'PRDCT5_CUSTM', 'PRDCT5_CUSTX', 'PRDCT5_CUSTY', 'PRDCT5_CUSTZ',
       'PRDCT6_CUSTK', 'PRDCT6_CUSTL', 'PRDCT6_CUSTM', 'PRDCT6_CUSTX',
       'PRDCT6_CUSTY', 'PRDCT6_CUSTZ', 'PRDCT7_CUSTK', 'PRDCT7_CUSTL',
       'PRDCT7_CUSTM', 'PRDCT7_CUSTX', 'PRDCT7_CUSTY', 'PRDCT7_CUSTZ',
       'PRDCT8_CUSTK', 'PRDCT8_CUSTL', 'PRDCT8_CUSTM', 'PRDCT8_CUSTX',
       'PRDCT8_CUSTY', 'PRDCT8_CUSTZ', 'PRDCT9_CUSTK', 'PRDCT9_CUSTL',
       'PRDCT9_CUSTM', 'PRDCT9_CUSTX', 'PRDCT9_CUSTY', 'PRDCT9_CUSTZ']

In [63]:
# ARIMA model
def arima_forecast(train, test):
    model = ARIMA(train, order=(1, 0, 0))  # ARIMA(p, d, q)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# SARIMA model
def sarima_forecast(train, test):
    model = SARIMAX(train, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))  # SARIMA(p, d, q)(P, D, Q, m)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# Holt-Winters model
def holt_winters_forecast(train, test):
    model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(len(test))
    return forecast

In [ ]:
best_model = None
best_mape = float('inf')
for i in timeseries_key_values:
    df_prod1 = df1[df1['timeseries_key']==i]
    print("\n")
    
    print("----------RESULT FOR",i,"----------")
    
    # Group data by 'calweek' and aggregate 'Quantity'
    ts_data = df_prod1.groupby('calweek')['Quantity'].sum()
    
    # Split data into training and testing sets
    train = ts_data.iloc[:130]
    test = ts_data.iloc[130:]
    
    # Perform forecasting using ARIMA
    arima_predictions = arima_forecast(train, test)

    # Perform forecasting using SARIMA
    sarima_predictions = sarima_forecast(train, test)

    # Perform forecasting using Holt-Winters
    holt_winters_predictions = holt_winters_forecast(train, test)
    
    arima_mape = mean_absolute_percentage_error(test,arima_predictions)*100
    sarima_mape = mean_absolute_percentage_error(test,sarima_predictions)*100
    holt_winters_mape = mean_absolute_percentage_error(test,holt_winters_predictions)*100
    print("MAPE value for arima model:",arima_mape)
    
    print("MAPE value for sarima model:",sarima_mape)
    
    print("MAPE value for holt_winters_predictions model:",holt_winters_mape)

    if arima_mape < best_mape:
        best_model = "ARIMA"
        best_mape = arima_mape
    if sarima_mape < best_mape:
        best_model = "SARIMA"
        best_mape = sarima_mape
    if holt_winters_mape < best_mape:
        best_model = "Holt-Winters"
        best_mape = holt_winters_mape

    print("Best model:", best_model)
    print("Best MAPE value:", best_mape)



----------RESULT FOR PRDCT1_CUSTA ----------
MAPE value for arima model: 3.259209679479494
MAPE value for sarima model: 12.611328260186342
MAPE value for holt_winters_predictions model: 3.268848846809093
Best model: ARIMA
Best MAPE value: 3.259209679479494


----------RESULT FOR PRDCT1_CUSTB ----------
MAPE value for arima model: 2.9216647631860626
MAPE value for sarima model: 4.6187088239710485
MAPE value for holt_winters_predictions model: 3.2095315436842498
Best model: ARIMA
Best MAPE value: 2.9216647631860626


----------RESULT FOR PRDCT1_CUSTC ----------


In [36]:
# Group data by 'calweek' and aggregate 'Quantity'
ts_data = df_prod1.groupby('calweek')['Quantity'].sum()

In [37]:
ts_data

calweek
202001    537
202002    544
202003    574
202004    523
202005    571
         ... 
202311    553
202312    522
202313    559
202314    541
202315    544
Name: Quantity, Length: 171, dtype: int64

In [38]:
len(ts_data)

171

In [39]:
# Split data into training and testing sets
train = ts_data.iloc[:130]
test = ts_data.iloc[130:]

In [40]:
# ARIMA model
def arima_forecast(train, test):
    model = ARIMA(train, order=(1, 0, 0))  # ARIMA(p, d, q)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# SARIMA model
def sarima_forecast(train, test):
    model = SARIMAX(train, order=(1, 0, 0), seasonal_order=(1, 0, 0, 12))  # SARIMA(p, d, q)(P, D, Q, m)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=len(test))
    return forecast

# Holt-Winters model
def holt_winters_forecast(train, test):
    model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(len(test))
    return forecast

In [41]:
# Perform forecasting using ARIMA
arima_predictions = arima_forecast(train, test)

# Perform forecasting using SARIMA
sarima_predictions = sarima_forecast(train, test)

# Perform forecasting using Holt-Winters
holt_winters_predictions = holt_winters_forecast(train, test)

In [42]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(test,arima_predictions)*100

3.259209679479494

In [43]:
mean_absolute_percentage_error(test,sarima_predictions)*100

12.611328260186342

In [44]:
mean_absolute_percentage_error(test,holt_winters_predictions)*100

3.268848846809093

In [45]:
# # Visualize the forecasts
# plt.plot(test.index, test, label='Actual')
# plt.plot(test.index, arima_predictions, label='ARIMA')
# plt.plot(test.index, sarima_predictions, label='SARIMA')
# plt.plot(test.index, holt_winters_predictions, label='Holt-Winters')

### FORECASTING

In [46]:
# Split data into training and testing sets
train = ts_data.iloc[:172]

In [47]:
# Holt-Winters model
def holt_winters_forecast(train):
    model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add')
    model_fit = model.fit()
    forecast = model_fit.forecast(13)
    return forecast

In [48]:
# Perform forecasting using Holt-Winters
holt_winters_predictions = holt_winters_forecast(train)

In [49]:
holt_winters_predictions

171    536.065287
172    541.988671
173    544.762172
174    542.621226
175    540.535257
176    545.423074
177    544.284717
178    547.920536
179    538.486300
180    542.522590
181    543.853003
182    533.396751
183    535.223393
dtype: float64